# Bursty Train Analysis

In [18]:
import os
import pandas as pd
import joblib
import numpy as np
from collections import Counter


#-----------------------------IMPORTANT-------------------------------------------
### VERY IMPORTANT: Changing the path from library to library2 change the folder of original results
import sys
sys.path.insert(0, '../Script/library3/')

from utilities import *
from randomization import *
from burst_func import *
import networkx as nx
from tqdm import tqdm

In [62]:
def burst_train(inter_ev,dt):
    ev_distr = np.zeros(len(inter_ev))
    i = 0
    
    cnt = Counter()
    d = 0
    c = 1
    ev_distr[0] = d
    while i < len(inter_ev):
        
        
        if inter_ev[i]<= dt:
            
            ev_distr[i+1] = d
            c+=1
            
            i +=1
    
        else:
            
            cnt.update(Counter([c]))
            
            d +=1
            
            if i<len(inter_ev)-1:ev_distr[i+1] = d
            
            i+=1
            c = 1
            
            continue
    assert (sum([k*v for k,v in cnt.items()]) == np.sum([k*v for k,v in Counter(Counter(ev_distr).values()).items()]))
    return cnt,ev_distr


def get_burst_train(df,dt):
    
    df_rest = df.copy()
    int_ev_time = [x for x in (df_rest.timestamp.shift(-1) - df_rest.timestamp).values if ((x!= np.nan))]
    b_train = burst_train(int_ev_time,dt)[0]
    bursts = burst_train(int_ev_time,dt)[1]
    df['burst'] = bursts
    return Counter(b_train),df,

### Compute Trains of egonetwork activity timer series for Randomized reference models

In [62]:
def compute_bursty_trains_rand(df,line_graph,dt_list,seed,title1):
    
    new_burst1 = {}
    neigh_new_burst1 = {}

    rand_orig_neigh_burst1 = {}
    rand_new_neigh_burst1 = {}



    n_link_rand_orig_neigh_burst1 = {}
    n_link_neigh_new_burst1 = {}
    n_link_rand_new_neigh_burst1 = {}

    df_new = permute_timestamps(df,seed)    ### timestamp reshuffle
    df1 = shuffle_df(df,seed)               ### timeseries reshuffle
    df1_n = random_df_same_weight(df,seed)  ### timeseries reshuffle with same n contacts
    for dt in dt_list:

        cnt1_s = Counter()

        cnt1_n = Counter()
        cnt_rn = Counter()
        cnt1_rn = Counter()


        cnt_rn_links = Counter()
        cnt1_n_links = Counter()
        cnt1_rn_links = Counter()

        for nodes in tqdm(df_new.nodes.unique()):

                neigh_set = (set(list(nx.neighbors(line_graph,nodes)))|set([nodes]))

                df_rest_new = df_new[df_new.nodes == nodes]

                df_rest_new_neigh = df_new[df_new.nodes.isin(neigh_set)] ### timestamp reshuffle
                df_rest_neigh_r = df1[df1.nodes.isin(neigh_set)]   ### timeseries reshuffle
                df_rest_new_neigh_r = df1_n[df1_n.nodes.isin(neigh_set)] ### timeseries reshuffle with same n contacts



                brst_cnt_new_neigh,df_rest_new_neigh = get_burst_train(df_rest_new_neigh,dt) ### timestamp reshuffle
                brst_cnt_neigh_r,df_rest_neigh_r = get_burst_train(df_rest_neigh_r,dt) ### timeseries reshuffle
                brst_cnt_new_neigh_r,df_rest_new_neigh_r = get_burst_train(df_rest_new_neigh_r,dt) ### timeseries reshuffle with same n contacts





                cnt1_s.update(get_burst_train(df_rest_new,dt=dt)[0])
                cnt1_n.update(brst_cnt_new_neigh)    ### timestamp reshuffle
                cnt_rn.update(brst_cnt_neigh_r)       ### timeseries reshuffle
                cnt1_rn.update(brst_cnt_new_neigh_r)   ### timeseries reshuffle with same n contacts






                cnt_rn_links.update((Counter(df_rest_neigh_r.groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x))))).values)))
                cnt1_n_links.update((Counter(df_rest_new_neigh.groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x))))).values)))
                cnt1_rn_links.update((Counter(df_rest_new_neigh_r.groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x))))).values)))




        new_burst1[title1+str(dt)] = cnt1_s
        neigh_new_burst1[title1+str(dt)] = cnt1_n      ### timestamp reshuffle
        rand_orig_neigh_burst1[title1+str(dt)] = cnt_rn  ### timeseries reshuffle
        rand_new_neigh_burst1[title1+str(dt)] = cnt1_rn   ### timeseries reshuffle with same n contacts



        n_link_rand_orig_neigh_burst1[title1+str(dt)] = cnt_rn_links            ### timeseries reshuffle
        n_link_neigh_new_burst1[title1+str(dt)] = cnt1_n_links                  ### timestamp reshuffle
        n_link_rand_new_neigh_burst1[title1+str(dt)] = cnt1_rn_links            ### timeseries reshuffle with same n contacts

        ### oreder: timeseries reshuffle, single, timestamp reshuffle, timeseries reshuffle with same n contacts
        file_2_save = map(lambda x: x[title1+str(dt)],[rand_orig_neigh_burst1,new_burst1,neigh_new_burst1,rand_new_neigh_burst1])
        ### oreder: timeseries reshuffle, timestamp reshuffle, timeseries reshuffle with same n contacts
        file_2_save_link = map(lambda x: x[title1+str(dt)],[n_link_rand_orig_neigh_burst1,n_link_neigh_new_burst1,n_link_rand_new_neigh_burst1])

        try: os.mkdir('../Results/Bursty_trains/'+title1+'rand/')
        except: print '../Results/Bursty_trains/'+title1+'rand/'
       
        a = '../Results/Bursty_trains/'+title1+'rand/'+title1+'_'+str(dt)+'_'+str(seed)
        #assert file_2_save == joblib.load(a+'.joblib')
        joblib.dump(file_2_save,a+'.joblib')

        b = '../Results/Bursty_trains/'+title1+'rand/'+title1+'_n_links_'+str(dt)+'_'+str(seed)
        #assert file_2_save_link == joblib.load(b+'.joblib')
        joblib.dump(file_2_save_link,b+'.joblib')

### Compute Trains of egonetwork activity timer series for Data

In [63]:
def compute_bursty_trains_data(df,line_graph,dt_list,title1):
    orig_burst1 = {}
    new_burst1 = {}
    neigh_new_burst1 = {}
    neigh_orig_burst1 = {}
    rand_orig_neigh_burst1 = {}
    rand_new_neigh_burst1 = {}

    n_link_neigh_new_burst1 = {}
    n_link_neigh_orig_burst1 = {}
    n_link_rand_orig_neigh_burst1 = {}
    n_link_rand_new_neigh_burst1 = {}
    
    #    if title1 !='tij_lnVS':continue
    for dt in dt_list:
        cnt_s = Counter()
        cnt1_s = Counter()
        cnt_n = Counter()
        cnt1_n = Counter()
        cnt_rn = Counter()
        cnt1_rn = Counter()

        cnt_n_links = Counter()
        cnt_rn_links = Counter()
        cnt1_n_links = Counter()
        cnt1_rn_links = Counter()
        k = 0
        for nodes in tqdm(df.nodes.unique()):

            neigh_set = (set(list(nx.neighbors(g,nodes)))|set([nodes]))
            df_rest = df[df.nodes == nodes]
            df_rest_neigh = df[df.nodes.isin(neigh_set)]                    ### data



            brst_cnt_neigh,df_rest_neigh = get_burst_train(df_rest_neigh,dt)





            cnt_s.update(get_burst_train(df_rest,dt)[0])
            cnt_n.update(brst_cnt_neigh)               ### data






            cnt_n_links.update((Counter(df_rest_neigh.groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x))))).values)))



            k += 1

        orig_burst1[title1+str(dt)] = cnt_s
        new_burst1[title1+str(dt)] = cnt1_s
        neigh_new_burst1[title1+str(dt)] = cnt1_n         ### permute timestamps
        neigh_orig_burst1[title1+str(dt)] = cnt_n         ### data
        rand_orig_neigh_burst1[title1+str(dt)] = cnt_rn   ### permute timeseries
        rand_new_neigh_burst1[title1+str(dt)] = cnt1_rn   ### permute timeseries preserve contacts



        n_link_neigh_orig_burst1[title1+str(dt)] = cnt_n_links
        n_link_rand_orig_neigh_burst1[title1+str(dt)] = cnt_rn_links
        n_link_neigh_new_burst1[title1+str(dt)] = cnt1_n_links
        n_link_rand_new_neigh_burst1[title1+str(dt)] = cnt1_rn_links


        file_2_save = map(lambda x: x[title1+str(dt)],[orig_burst1,neigh_orig_burst1,rand_orig_neigh_burst1,new_burst1,neigh_new_burst1,rand_new_neigh_burst1])
        file_2_save_link = map(lambda x: x[title1+str(dt)],[n_link_neigh_orig_burst1,n_link_rand_orig_neigh_burst1,n_link_neigh_new_burst1,n_link_rand_new_neigh_burst1])


        try: os.mkdir('../Results/Bursty_trains/'+title1+'/')
        except: print '../Results/Bursty_trains/'+title1+'/'
       
        a = '../Results/Bursty_trains/'+title1+'/'+title1+'_'+str(dt)
        #assert file_2_save == joblib.load(a+'.joblib')
        joblib.dump(file_2_save,a+'.joblib')

        b = '../Results/Bursty_trains/'+title1+'/'+title1+'_n_links_'+str(dt)
        #assert file_2_save_link == joblib.load(b+'.joblib')
        joblib.dump(file_2_save_link,b+'.joblib')

    




### Compute Trains for time series of activity of top 10% most active links for Data

In [64]:

def compute_bursty_trains_most_active(df,line_graph,dt_list,title1): 
    for dt in [60,120,300,600,1200,2400,3600]:

        cnt_tot = {}
        link_cnt_tot = {}

    #    if title1 !='tij_lnVS': continue
    #     if title1 in rand_new_neigh_burst1.keys():continue
            #node_list = list(g.nodes)
        node_list1 = df.groupby(df.nodes).size().sort_values(ascending = False)[:int(float(len(g.nodes))*0.1)].index
        node_rank = df.groupby(df.nodes).size()
        minimum_value = node_rank.sort_values(ascending = False)[:int(float(len(g.nodes))*0.1)].iloc[-1]
        node_list2 = node_rank[node_rank>=minimum_value].index

        df_list = list()

        cnt_n = Counter()
        cnt1_n_links = Counter()



        k = 0
        for nodes in tqdm(node_list2):


            df_rest_neigh = df[df.nodes == nodes]   


            brst_cnt_neigh,df_rest_neigh = get_burst_train(df_rest_neigh,dt)






            cnt_n.update(brst_cnt_neigh)               ### data



        cnt_tot[title1] = cnt_n
        try:os.mkdir('../Results/Bursty_trains_most_active/')
        except: print 'folder already existing'
        #assert cnt_tot == joblib.load('../Results/Bursty_trains_most_active/'+title1+'_'+str(dt)+'.joblib')       
        joblib.dump(cnt_tot,'../Results/Bursty_trains_most_active/'+title1+'_'+str(dt)+'.joblib')       


### Example

In [66]:
seed = 1
dt_list = [60,120,300,600,1200,2400,3600]
print seed


title1 = 'tij_lnVS'
df = get_df(title1)
g = get_linegraph(title1)
    
compute_bursty_trains_rand(df,g,dt_list,seed,title1)

compute_bursty_trains_data(df,g,dt_list,title1)

compute_bursty_trains_most_active(df,g,dt_list,title1)

1
